# Minimal FaceGreeter Flask Test

This notebook tests a basic Flask server in Google Colab to debug kernel crashes. It runs a simple Flask app with a `/hello` endpoint, without git clone, ngrok, or external dependencies beyond Flask. If stable, we'll add ngrok, git, and video processing.

## Prerequisites
- Google Colab environment with internet access.

## Steps
1. Install Flask.
2. Create and run a basic Flask app.
3. Monitor resources and check for kernel crashes.

**Goal**: Confirm Flask runs without kernel crashes.

In [ ]:
# Install Flask and monitor resources
import os
import psutil

# Log initial resource usage
print(f'Initial RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Initial CPU: {psutil.cpu_percent()}%')

# Purge pip cache
!pip cache purge

# Install Flask
!pip install --no-cache-dir flask==3.0.3

# Verify installed packages
!pip list | grep flask

# Log resource usage after install
print(f'Post-install RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Post-install CPU: {psutil.cpu_percent()}%')

print('Flask installed. Proceed to the next cell to run the Flask app.')

In [ ]:
# Run minimal Flask app
from flask import Flask, jsonify
import os
import psutil
import threading

# Log resource usage before Flask
print(f'Pre-Flask RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Pre-Flask CPU: {psutil.cpu_percent()}%')

# Create Flask app
app = Flask(__name__)

@app.route('/hello', methods=['GET'])
def hello():
    return jsonify({"success": True, "message": "Hello from Flask!"})

# Run Flask in a separate thread to avoid blocking the notebook
def run_flask():
    print('Starting Flask server on http://0.0.0.0:8000')
    app.run(host='0.0.0.0', port=8000, debug=False, use_reloader=False)

print('Starting Flask server...')
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

# Wait briefly to ensure Flask starts
import time
time.sleep(5)

# Check running processes and port
os.system('ps aux | grep python')
os.system('netstat -tuln | grep 8000')

# Log resource usage after Flask
print(f'Post-Flask RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Post-Flask CPU: {psutil.cpu_percent()}%')

# Keep the notebook running
print('Flask server running. Open http://<colab-ip>:8000/hello in a browser if accessible.')
print('Press Ctrl+C in this cell to stop the Flask server.')
try:
    flask_thread.join()
except KeyboardInterrupt:
    print('Stopping Flask server...')
    os.system('pkill python')
    os.system('fuser -k 8000/tcp')